In [29]:
import import_ipynb
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
import lenet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchvision.datasets import KMNIST
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
# from lenet import Test

In [30]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# load the KMNIST dataset
print("[INFO] loading the custom dataset...")
# trainData = KMNIST(root="data", train=True, download=True, transform=ToTensor())
# testData = KMNIST(root="data", train=False, download=True, transform=ToTensor())
transform = transforms.Compose([
    # resize
    #transforms.Resize(32),
    # center-crop
    #transforms.CenterCrop(32),
    # to-tensor
    transforms.ToTensor(),
    # normalize
    #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
dataset = ImageFolder("images", transform)
trainData, valData, testData = random_split(dataset, [180, 30, 30])
# calculate the train/validation split
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData) * VAL_SPLIT)
(trainData, valData) = random_split(trainData,
	[numTrainSamples, numValSamples],
	generator=torch.Generator().manual_seed(42))

[INFO] loading the custom dataset...
[INFO] generating the train/validation split...


In [32]:
# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainData, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [33]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = lenet.LeNet(
	numChannels=3,
	classes=5).to(device)
    #len(trainData.dataset.classes)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] initializing the LeNet model...
[INFO] training the network...


In [34]:
# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()

	# switch off autograd for evaluation
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in valDataLoader:
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFn(pred, y)
			# calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()

	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avgValLoss, valCorrect))

torch.Size([64, 20, 396, 396])
torch.Size([64, 20, 396, 396])
torch.Size([7, 20, 396, 396])
torch.Size([45, 20, 396, 396])
[INFO] EPOCH: 1/10
Train loss: 39.172794, Train accuracy: 0.2741
Val loss: inf, Val accuracy: 0.1333

torch.Size([64, 20, 396, 396])
torch.Size([64, 20, 396, 396])
torch.Size([7, 20, 396, 396])
torch.Size([45, 20, 396, 396])
[INFO] EPOCH: 2/10
Train loss: 9.873906, Train accuracy: 0.2000
Val loss: inf, Val accuracy: 0.2222

torch.Size([64, 20, 396, 396])
torch.Size([64, 20, 396, 396])
torch.Size([7, 20, 396, 396])
torch.Size([45, 20, 396, 396])
[INFO] EPOCH: 3/10
Train loss: 2.023755, Train accuracy: 0.4074
Val loss: inf, Val accuracy: 0.2444

torch.Size([64, 20, 396, 396])
torch.Size([64, 20, 396, 396])
torch.Size([7, 20, 396, 396])
torch.Size([45, 20, 396, 396])
[INFO] EPOCH: 4/10
Train loss: 2.275004, Train accuracy: 0.4370
Val loss: inf, Val accuracy: 0.4444

torch.Size([64, 20, 396, 396])
torch.Size([64, 20, 396, 396])
torch.Size([7, 20, 396, 396])
torch.Size(

In [45]:
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
print(len(trainDataLoader))
print(len(valDataLoader))
print(len(testDataLoader))
with torch.no_grad():
	# set the model in evaluation mode
	model.eval()
	
	# initialize a list to store our predictions
	#preds = []
	# loop over the test set
	for (x, y) in testDataLoader:
		preds = []        
		# send the input to the device
		x = x.to(device)
		# make the predictions and add them to the list
		pred = model(x)
		print(pred)
		preds.extend(pred.argmax(axis=1).cpu().numpy())
		print(preds)
		print(y)
		confusionMatrix = np.zeros((5, 5))
		#print(confusionMatrix)
		counter = 0        
		for tv in y:
			#print(counter)
			#print(tv.item())
			prediction = preds[counter]
			#print("prediction is " + str(prediction))
			#print("true value is " + str(tv.item()))
			confusionMatrix[prediction][tv.item()] += 1
			counter += 1
print(confusionMatrix)
'''
			if prediction == 0 and tv.item() == 0:
				confusionMatrix[0][0] += 1
				#print("adding a correct guess to category 1")                
			elif prediction == 0 and tv.item() != 0:
				confusionMatrix[0][1] += 1
				#print("adding a incorrect guess to category 1")  
			elif prediction == 1 and tv.item() == 1:
				confusionMatrix[1][0] += 1
				#print("adding a correct guess to category 2")  
			elif prediction == 1 and tv.item() != 1:
				confusionMatrix[1][1] += 1
				#print("adding a incorrect guess to category 2")  
			elif prediction == 2 and tv.item() == 2:
				confusionMatrix[2][0] += 1
				#print("adding a correct guess to category 3")  
			elif prediction == 2 and tv.item() != 2:
				confusionMatrix[2][1] += 1
				#print("adding a incorrect guess to category 3")  
			elif prediction == 3 and tv.item() == 3:
				confusionMatrix[3][0] += 1
				#print("adding a correct guess to category 4")  
			elif prediction == 3 and tv.item() != 3:
				confusionMatrix[3][1] += 1
				#print("adding a incorrect guess to category 4")  
			elif prediction == 4 and tv.item() == 4:
				confusionMatrix[4][0] += 1
				#print("adding a correct guess to category 5")  
			elif prediction == 4 and tv.item() != 4:
				confusionMatrix[4][1] += 1
				#print("adding a incorrect guess to category 5")  
'''
# generate a classification report
# print(classification_report(testData.targets.cpu().numpy(),
#	np.array(preds), target_names=testData.classes))

[INFO] total time taken to train the model: 1377.29s
[INFO] evaluating network...
3
1
1
torch.Size([30, 20, 396, 396])
tensor([[-1.0937e+01, -7.4353e+00, -1.1761e-02, -1.0981e+01, -4.5038e+00],
        [-8.2896e-01, -1.0992e+01, -5.7540e-01, -6.9074e+00, -1.8423e+01],
        [-2.8015e+00, -3.4261e-01, -2.1412e+00, -4.3656e+00, -2.3111e+00],
        [-1.0272e+01, -8.5872e+00, -3.6005e-03, -6.0731e+00, -6.8410e+00],
        [-1.0344e+01, -1.3142e+01, -5.6663e+00, -4.2426e-03, -7.2106e+00],
        [-3.2662e+00, -2.2662e-01, -1.8216e+00, -9.7609e+00, -5.8771e+00],
        [-6.3577e+01, -2.0179e+01,  0.0000e+00, -7.1791e+01, -6.1602e+01],
        [-2.3001e+00, -1.2166e+00, -1.0331e+00, -1.6476e+00, -2.8984e+00],
        [-3.8339e-01, -1.2601e+00, -5.4957e+00, -4.3940e+00, -3.9977e+00],
        [-2.1817e+01, -1.4770e+01, -3.5763e-07, -2.4177e+01, -3.7791e+01],
        [-9.5814e+00, -5.0846e+00, -2.1847e-01, -1.2538e+01, -1.6608e+00],
        [-1.9541e+01, -6.4700e+00, -1.5508e-03, -1.5281e

'\n\t\t\tif prediction == 0 and tv.item() == 0:\n\t\t\t\tconfusionMatrix[0][0] += 1\n\t\t\t\t#print("adding a correct guess to category 1")                \n\t\t\telif prediction == 0 and tv.item() != 0:\n\t\t\t\tconfusionMatrix[0][1] += 1\n\t\t\t\t#print("adding a incorrect guess to category 1")  \n\t\t\telif prediction == 1 and tv.item() == 1:\n\t\t\t\tconfusionMatrix[1][0] += 1\n\t\t\t\t#print("adding a correct guess to category 2")  \n\t\t\telif prediction == 1 and tv.item() != 1:\n\t\t\t\tconfusionMatrix[1][1] += 1\n\t\t\t\t#print("adding a incorrect guess to category 2")  \n\t\t\telif prediction == 2 and tv.item() == 2:\n\t\t\t\tconfusionMatrix[2][0] += 1\n\t\t\t\t#print("adding a correct guess to category 3")  \n\t\t\telif prediction == 2 and tv.item() != 2:\n\t\t\t\tconfusionMatrix[2][1] += 1\n\t\t\t\t#print("adding a incorrect guess to category 3")  \n\t\t\telif prediction == 3 and tv.item() == 3:\n\t\t\t\tconfusionMatrix[3][0] += 1\n\t\t\t\t#print("adding a correct guess to c

In [9]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.jpg")
# serialize the model to disk
torch.save(model, "model.h5")